# 🦠 Análisis de datos de secuenciación de amplicones con QIIME 2

### 🔗 Links Importantes
- [Course Github](https://github.com/Gibbons-Lab/isb_course_2024): Repositorio de Github para el MSHA 2025
    
- [QIIME 2 view](http://view.qiime2.org): Una interfaz basada en navegador para ver artefactos y visualizaciones de QIIME (tablas y gráficos)
- [QIIME 2 plugins](https://docs.qiime2.org/2024.5/plugins/): Más información sobre los complementos QIIME 2 que utilizamos hoy

### 🛑 STOP! Antes de ejecutar cualquier cosa...

1. Guarda una copia local de este cuaderno en `File > Save a copy in Drive`. En algún momento, es posible que se te solicite que confíes en el cuaderno. Te garantizamos que es seguro. 🤞

2. Ten en cuenta que **el código de este cuaderno debe ejecutarse EN ORDEN** si te pierdes o encuentras errores. Favor de revisarlo con los miembros del staff del curso.

**Descargo de responsabilidad:** El entorno de notebook de Google Colab interpretará cualquier comando como código Python por defecto. Si queremos ejecutar comandos bash, tendremos que anteponerles `!`. Por lo tanto, cualquier comando que vea con `!` inicial es un comando bash y, si quisiera ejecutarlo en su terminal, omitiría el `!`. Por ejemplo, si en el notebook de Colab ejecuta `!wget`, simplemente ejecutaría `wget` en su terminal.

---
# Setup

QIIME 2 suele instalarse siguiendo las [instrucciones oficiales de instalación](https://docs.qiime2.org/2024.5/install/). Sin embargo, dado que usamos Google Colab y el uso de conda tiene algunas limitaciones, tendremos que modificar un poco la instalación. Pero no se preocupen, a continuación encontrarán un script de instalación que realiza todo el trabajo por nosotros 😌


Comencemos descargando una copia local del repositorio del proyecto desde GitHub. Este repositorio, llamado `materials`, contiene todos los datos relevantes y otros recursos que necesitaremos para este taller.

In [1]:
!git clone https://github.com/diegoibt/microbiome_16S materials

Cloning into 'materials'...
remote: Enumerating objects: 90, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 90 (delta 4), reused 11 (delta 3), pack-reused 75 (from 1)
Receiving objects: 100% (90/90), 86.44 MiB | 35.56 MiB/s, done.
Resolving deltas: 100% (19/19), done.


Para ver el directorio, haga clic en el icono de carpeta 📁 a la izquierda. Para ejecutar código desde este directorio, acceda a él mediante la línea de comandos:

In [3]:
%cd materials

/content/materials


Observe que aquí usamos ```%``` en lugar de ```!``` para ejecutar nuestra función de línea de comandos. Esto hace que la ruta a nuestro directorio sea permanente. El operador ```!``` solo cambia el intérprete para que espere indicaciones de línea de comandos temporalmente.



Ahora que tenemos todos los materiales, estamos casi listos para empezar, aunque aún no del todo. ¿Recuerdan QIIME? Necesitamos instalarlo antes de comenzar el análisis. No se preocupen, esto solo se configurará en el notebook de Colab, no en su equipo local.

**Ejecutemos la siguiente celda para instalar y configurar QIIME2**

In [ ]:
%run installqiime2_prueba.py

⬆️ Esto llevará un tiempo (normalmente de 10 a 15 minutos), así que volveremos a la **presentación** mientras esperamos.

Si quieres saber más sobre QIIME2, te recomendamos consultar la [documentación](https://docs.qiime2.org/). También te explicará cómo instalar QIIME2 en tu equipo local. 🖥

---
# Importación de datos

¡QIIME2 puede llevarnos desde secuencias sin procesar al conocimiento ecológico!
![our workflow](https://github.com/Gibbons-Lab/isb_course_2024/raw/main/docs/16S/assets/steps.png)
Pero primero necesitamos transferir nuestros datos sin procesar a QIIME2. Para ello, necesitaremos:
1. **Archivos de secuenciación sin procesar** (.fastq): Los archivos FASTQ contienen puntuaciones de secuencia y calidad, a diferencia de los archivos FASTA, que solo contienen la secuencia. Necesitamos estas puntuaciones de calidad para poder recortar o descartar lecturas de baja calidad.

2. **Archivo de metadatos** (.tsv): Este archivo contiene el ID de la muestra, junto con todos los datos no secuenciales de cada muestra. Aquí, nuestros metadatos incluyen el estado de la enfermedad, la edad, el sexo, el IMC, la ubicación geográfica y el uso de medicamentos recetados, así como información sobre la dieta y los síntomas gastrointestinales.

3. **Archivo de manifiesto** (.tsv): Este archivo contiene el ID de la muestra y la ruta a su archivo de secuenciación. Así es como QIIME asocia una secuencia con sus metadatos correctos. 📝

Comencemos analizando nuestros datos. En la carpeta de `data`, encontrará 10 archivos FASTQ, un manifiesto de archivo y un archivo de metadatos.

Si leemos `manifest.tsv`, veremos que es una tabla que contiene el nombre y la ruta de archivo de todas nuestras muestras.

In [5]:
import pandas as pd
manifest = pd.read_csv('data/manifest.tsv', sep = '\t')
manifest

,sample-id,absolute-filepath
0,ERR1513701,$PWD/data/ERR1513701.fastq.gz
1,ERR1513870,$PWD/data/ERR1513870.fastq.gz
2,ERR1513889,$PWD/data/ERR1513889.fastq.gz
3,ERR1513684,$PWD/data/ERR1513684.fastq.gz
4,ERR1513703,$PWD/data/ERR1513703.fastq.gz
5,ERR1514003,$PWD/data/ERR1514003.fastq.gz
6,ERR1513961,$PWD/data/ERR1513961.fastq.gz
7,ERR1513983,$PWD/data/ERR1513983.fastq.gz
8,ERR1513964,$PWD/data/ERR1513964.fastq.gz
9,ERR1513777,$PWD/data/ERR1513777.fastq.gz


También podemos consultar `metadata.tsv`, que nos dará más contexto sobre nuestras muestras 🔬

In [ ]:
metadata = pd.read_csv('data/metadata.tsv', sep='\t')
metadata

Todo bien, los 10 archivos FASTQ están contabilizados: cinco sanos y cinco con enfermedad de Parkinson. Podemos usar el archivo de manifiesto para importar nuestros archivos a QIIME.

## FASTQ a Artefacto

Para usar datos de secuenciación en QIIME, primero debemos convertir los archivos FASTQ que contienen nuestros datos en artefactos QIIME. Usando el manifiesto que acabamos de consultar, ejecutemos nuestro primer comando:

-- Como recordatorio, añadir ```!``` antes del comando indica al notebook que se trata de un comando de **bash**, no de Python.

In [ ]:
!qiime tools import \
  --type 'SampleData[SequencesWithQuality]' \
  --input-path data/manifest.tsv \
  --output-path sequences.qza \
  --input-format SingleEndFastqManifestPhred33V2

Analicemos el comando QIIME. Los comandos QIIME tienen el siguiente formato:

```
qiime plugin action --i-argument1 ... --o-argument2 ...
```
En el comando anterior, llamamos al complemento ```tools``` de QIIME2 para importar nuestros datos. Los siguientes argumentos indican dónde se encuentra el manifiesto y dónde se debe guardar la salida. También le indicamos a QIIME qué tipo de entrada esperar.

Los tipos de argumentos suelen comenzar con una letra que indica su significado:

- `--i-...` = archivos de entrada
- `--o-...` = archivos de salida
- `--p-...` = parámetros
- `--m-...` = metadatos



## Artefacto para la visualización 🔎

Antes de continuar, usemos QIIME para visualizar nuestros datos de secuenciación.

In [ ]:
!qiime demux summarize \
--i-data sequences.qza \
--o-visualization qualities.qzv

Los archivos .qzv como el que acabamos de producir son de visualización. Puede ver el gráfico **descargando el archivo y abriéndolo en http://view.qiime2.org**. Para descargarlo, haga clic en el símbolo de carpeta a la izquierda, abra la carpeta `materials` y seleccione `download` en el menú desplegable junto al archivo `quality.qzv`.

---
# Filtrado de calidad: de la secuencia al ASV

Antes de poder usar nuestros datos de secuenciación, necesitamos eliminar el ruido. Para ello, usaremos un complemento llamado ```DADA2```. Esto implica lo siguiente:

1. Filtrar y recortar las lecturas
2. Encontrar el conjunto más probable de secuencias únicas en la muestra (ASV)
3. Eliminar quimeras
4. Contar la abundancia de cada ASV

Este comando tardará un poco; ejecutémoslo y volvamos a la presentación para analizar lo que está sucediendo.

In [ ]:
!qiime dada2 denoise-single \
    --i-demultiplexed-seqs sequences.qza \
    --p-trunc-len 150 \
    --p-n-threads 2 \
    --output-dir dada --verbose

Si abrimos la carpeta `dada` 📁, veremos varios resultados.
- `representative_sequences.qza` es nuestra *tabla de abundancia ASV* 🧬
- `denoising-stats.qza` contiene las *estadísticas de reducción de ruido* 📊

Queremos revisar las estadísticas de reducción de ruido para asegurarnos de que la eliminación de ruido se realizó correctamente. Es normal perder entre un 5 y un 25 % de las lecturas debido a quimeras, pero una pérdida importante podría deberse a ejecutar demasiados ciclos de PCR.

Como se mencionó anteriormente, no podemos ver los *artefactos* de QIIME directamente, pero podemos convertirlos en *visualizaciones* mediante la acción `tabulate` del complemento `metadata`.

In [ ]:
!qiime metadata tabulate \
    --m-input-file dada/denoising_stats.qza \
    --o-visualization dada/denoising-stats.qzv

Como antes, podemos descargar el archivo .qzv y visualizar los resultados con el [QIIME2 Viewer](https://view.qiime2.org/).

Es importante comprender qué nos indica este resultado. Por ejemplo, ¿qué porcentaje de lecturas de nuestros datos supera el paso de filtrado? ¿Qué porcentaje de lecturas no fueron quiméricas? Las diferencias en estas métricas entre muestras pueden afectar las métricas de diversidad.



Para obtener más información sobre el filtrado de calidad pair-end **consulta**:

Dacey, Daniel P., and Frédéric JJ Chain. "Concatenation of paired-end reads improves taxonomic classification of amplicons for profiling microbial communities." BMC bioinformatics 22 (2021): 1-25.




---
# Filogenética

Ahora que tenemos una tabla ASV, ¡podemos calcular métricas de diversidad! Pero primero, regresen a la **presentación**

Nota: Estamos abordando análisis que algunos prefieren realizar en RStudio o Jupyter Notebooks, sin usar el framework QIIME. Por ejemplo, los análisis de diversidad se pueden realizar usando `scikit-bio` en Python o `vegan` en R. De hecho, QIIME2 utiliza `scikit-bio` y `vegan` internamente para sus cálculos de diversidad.

### De ASV a árboles filogenéticos con QIIME2 `phylogeny`

Algunas *métricas de diversidad* se calculan mediante la distancia filogenética, así que comencemos construyendo un árbol filogenético para nuestras secuencias con el siguiente comando. En esta ocasión, llamamos al complemento `phylogeny` en QIIME2.

In [ ]:
!qiime phylogeny align-to-tree-mafft-fasttree \
    --i-sequences dada/representative_sequences.qza \
    --output-dir tree

Podemos crear una visualización para el árbol usando el complemento QIIME 2 `empress`. Consulte la [documentación](https://github.com/biocore/empress) para obtener más detalles.

In [ ]:
!qiime empress tree-plot \
    --i-tree tree/rooted_tree.qza \
    --o-visualization tree/empress.qzv

---
# Diversidad

## Parte 1: Los básicos

El complemento `diversity` de QIIME2 incluye una acción llamada `core-metrics-phylogenetic`, que calcula _varias_ métricas de diversidad filogenéticas y no filogenéticas.

Podemos usar nuestra tabla y árbol para calcular varias métricas de diversidad. Para tener en cuenta las variaciones en la **profundidad de muestreo**, proporcionaremos a QIIME2 un valor de corte en el que **rarificaremos** todas nuestras muestras. La rarefacción selecciona secuencias **aleatoriamente**, por lo que sus resultados podrían diferir ligeramente de los míos. También pasaremos nuestro archivo de metadatos para poder rastrear qué muestras provienen de cada grupo.

In [ ]:
!qiime diversity core-metrics-phylogenetic \
    --i-table dada/table.qza \
    --i-phylogeny tree/rooted_tree.qza \
    --p-sampling-depth 5000 \
    --m-metadata-file data/metadata.tsv \
    --output-dir diversity

Si abres la carpeta "diversidad", verás que calculamos todas las métricas de diversidad que acabo de mencionar. Para más información sobre cómo se calculan e interpretan estas métricas de diversidad, consulta esta publicación del foro de QIIME (https://forum.qiime2.org/t/alpha-and-beta-diversity-explanations-and-commands/2282).

Pero lo que realmente nos interesa es si la diversidad difiere *significativamente* entre muestras de pacientes con enfermedad y de pacientes sanos, o si existen otras covariables. Por lo tanto, necesitaremos realizar pruebas estadísticas.

Vuelve a la presentación para ver algunos tipos de pruebas estadísticas y cuándo usarlas.

### Diversidad alfa: riqueza y uniformidad _dentro_ de una muestra

Obtenemos varios resultados del comando anterior: medidas de diversidad alfa y beta. Para empezar, usemos el vector de Shannon del directorio de salida para crear una visualización de la diversidad alfa en las muestras. Generalmente, las personas sanas y longevas tienen microbiomas diversos y equilibrados. Sin embargo, esto no es necesariamente un indicador directo de salud o enfermedad. Veamos cómo se ve en nuestras muestras.

In [ ]:
!qiime diversity alpha-group-significance \
    --i-alpha-diversity diversity/shannon_vector.qza \
    --m-metadata-file data/metadata.tsv \
    --o-visualization diversity/alpha_groups.qzv

Al igual que antes, podemos descargar la visualización y abrirla con el [visor QIIME2](http://view.qiime2.org).

### Diversidad beta: distancia entre muestras

Visualicemos la diversidad beta y veamos cómo se separan. Para ello, analizaremos el UniFrac ponderado. En esta ocasión, tendremos que descargar el archivo ⬅️

Podemos comprobar si hay una separación significativa entre muestras usando PERMANOVA. Podemos hacerlo con el complemento de diversidad de QIIME2.

In [ ]:
!qiime diversity adonis \
    --i-distance-matrix diversity/weighted_unifrac_distance_matrix.qza \
    --m-metadata-file data/metadata.tsv \
    --p-formula "parkinson_disease" \
    --p-n-jobs 2 \
    --o-visualization diversity/permanova.qzv

Ya ejecutamos un PCoAs para cada métrica de distancia y podemos verlos si abrimos `weighted_unifrac_emperor.qzv`, `unweighted_unifrac_emperor.qzv`, `bray_curtis_emperor.qzv` o `jaccard_emperor.qzv` en el [visor QIIME2](http://view.qiime2.org).

### ¿Son informativos estos resultados? Un análisis del tamaño de la muestra, el tamaño del efecto y los factores de confusión.

Estos resultados no son muy informativos y, en algunos casos, contradicen la intuición. Esto puede deberse a varias razones:

1. **Tamaño muestral pequeño:** Utilizamos un tamaño muestral muy pequeño (n = 10). Incluso si pudiéramos observar tendencias, no esperaríamos significación estadística.

2. **La diversidad alfa es una métrica de grano grueso:** El índice de Shannon intenta resumir la diversidad del microbioma (muy compleja) en una sola cifra.

3. **Factores de confusión importantes:** Estos datos provienen de un estudio observacional, donde no se controlaron los factores de confusión. Se sabe que algunos factores de confusión (como la dieta y la exposición a fármacos) tienen fuertes efectos sobre la diversidad, lo que puede ocultar relaciones más pequeñas entre el microbioma y una variable de interés (como el estado de la enfermedad).

En los estudios observacionales, es común **estratificar** los factores de confusión o incluirlos como **covariables** en las regresiones. Sin embargo, con un tamaño muestral de n = 10, realmente solo podemos evaluar una variable sin romper los supuestos. **¿Pero qué pasaría si tuviéramos una muestra más grande?** Esto abriría la puerta a más tipos de análisis. No pudimos procesar todo el conjunto de datos en Colab debido a la capacidad computacional limitada, pero por suerte, **¡preprocesamos todo el conjunto de datos para ti!**

## Parte 2: Cómo lidiar con los factores de confusión

### Diversidad Alfa: conjunto de datos completo

Los archivos completos del conjunto de datos Hill-Burns 2017 se encuentran en la carpeta MATERIALES. Ahora, carguémoslos en QIIME2.

Mantendremos este análisis en la carpeta `complete_study` para no confundir los archivos anteriores.


Ahora tenemos muchos más metadatos que antes. Echemos un vistazo.

In [6]:
complete_metadata = pd.read_csv('complete_study/complete_metadata.tsv', sep='\t')
print(complete_metadata.shape)
complete_metadata.head()

(330, 39)


,id,parkinson_disease,sex,age,bmi,location,anticholinergic,antibiotics_bool,anti_inflammatory_bool,comt_inhibitor,...,crohns_disease,yogurt_live_culture,nuts,grains,fruits_or_vegetables,meats,tobacco,marijuana,alcohol_bool,coffee_bool
0,ERR1513669,No,female,79.0,28.34,"Atlanta, GA",N,No,Yes,N,...,No,NaN,Few times a week,Few times a week,Few times a week,Few times a week,False,False,True,True
1,ERR1513670,No,female,70.0,33.12,"Atlanta, GA",N,No,Yes,N,...,No,NaN,Few times a month,At least once a day,At least once a day,At least once a day,False,False,False,True
2,ERR1513671,No,female,79.0,32.28,"Atlanta, GA",N,No,No,N,...,No,Yes,At least once a day,At least once a day,At least once a day,At least once a day,False,False,False,False
3,ERR1513672,No,female,70.0,24.44,"Atlanta, GA",N,No,No,N,...,No,No,Few times a week,At least once a day,Few times a week,At least once a day,False,False,True,False
4,ERR1513673,No,male,76.0,25.06,"Atlanta, GA",N,No,Yes,N,...,No,NaN,At least once a day,Few times a week,At least once a day,Few times a week,False,False,True,True


En el artículo original, los autores hallaron que los medicamentos anticolinérgicos, los inhibidores de la catecol-O-metiltransferasa (COMT), la ubicación, el sexo, la edad y la EP tenían señales significativas e independientes sobre la composición del microbioma. ¿Cómo varía la diversidad alfa en función de estas variables?

In [ ]:
!qiime diversity alpha-group-significance \
    --i-alpha-diversity complete_study/diversity/shannon_vector.qza \
    --m-metadata-file complete_study/complete_metadata.tsv \
    --o-visualization complete_study/diversity/alpha_groups.qzv

Nuevamente, visualice en el [visor QIIME2](http://view.qiime2.org).

Observe que `alpha-group-significance` realiza una prueba de Kruskal-Wallis _separada_ para cada variable y genera un valor p _sin ajustar_. Si queremos analizar _múltiples_ relaciones entre la diversidad alfa y las covariables (es decir, realizar múltiples pruebas), necesitamos _controlar la tasa de falsos descubrimientos (FDR false discovery rate) corrigiendo nuestros valores p_. Hablaremos de esto más adelante.

### PCoA + PERMANOVA para diversidad beta

Como ahora tenemos n = 300+, podemos incluir más variables en nuestra PERMANOVA.

Primero, ejecutemos PERMANOVA solo para el estado de la enfermedad para que podamos ver realmente qué hace la adición de covariables en nuestros resultados.

In [ ]:
!qiime diversity adonis \
    --i-distance-matrix complete_study/diversity/weighted_unifrac_distance_matrix.qza \
    --m-metadata-file complete_study/complete_metadata.tsv \
    --p-formula "parkinson_disease" \
    --p-n-jobs 2 \
    --o-visualization complete_study/diversity/permanova_PD.qzv

Si observa la tabla en el [visor QIIME2](http://view.qiime2.org), notará que, dado que el tamaño de nuestra muestra aumentó, el efecto de PD en la diversidad beta de UniFrac (aunque muy pequeño en R cuadrado = 0,01) ahora es significativo (al menos antes de las correcciones FDR).

También podemos usar PERMANOVA para identificar factores de confusión. PERMANOVA nos indica cuánta varianza en la composición de la comunidad explica cada variable. Los factores de confusión comunes incluyen el sexo, la edad, el IMC, la dieta y el uso de antibióticos. En el estudio original, los autores identificaron que los medicamentos para la enfermedad de Parkinson se asociaban con diferentes composiciones del microbioma. Analicemos estas variables.

Let's look at the metadata file to see which variables have NaN values.

Pero espere: `adonis` requiere que las variables de la fórmula no tengan valores NaN ni valores faltantes. Si intentáramos ejecutar `adonis` con todas estas variables, generaría un error. Por lo tanto, tendremos que **filtrar** las muestras con valores NA para las variables que queramos probar.

Revisemos el archivo de metadatos para ver qué variables tienen valores NaN.

In [ ]:
metadata_perm = complete_metadata[['parkinson_disease', 'sex', 'age', 'bmi', 'comt_inhibitor', 'carbidopa_levodopa', 'anticholinergic', 'location']].copy()
metadata_perm

In [ ]:
# ¿Qué variables tienen valores N/A para la PERMANOVA que queremos ejecutar?
col_na_num = metadata_perm.isna().sum()
col_na = col_na_num[col_na_num > 0]
col_na

Parece que un subconjunto de personas no respondió estas preguntas. ¿Cuántas muestras tendremos si las descartamos?

In [ ]:
metadata_perm.dropna(axis=0, subset=['bmi', 'comt_inhibitor', 'carbidopa_levodopa', 'anticholinergic'], how='any')

Esta vez podemos usar la acción QIIME `filter-distance-matrix`

In [ ]:
# filter QIIME table
!qiime diversity filter-distance-matrix \
    --i-distance-matrix complete_study/diversity/weighted_unifrac_distance_matrix.qza \
    --m-metadata-file complete_study/complete_metadata.tsv \
    --p-where "[bmi] IS NOT NULL AND [comt_inhibitor] IS NOT NULL AND [carbidopa_levodopa] IS NOT NULL AND [anticholinergic] IS NOT NULL" \
    --o-filtered-distance-matrix complete_study/diversity/weighted_unifrac_distance_matrix_no_NA.qza

In [ ]:
!qiime diversity adonis \
    --i-distance-matrix complete_study/diversity/weighted_unifrac_distance_matrix_no_NA.qza \
    --m-metadata-file complete_study/complete_metadata.tsv \
    --p-formula "parkinson_disease + sex + age + bmi + location + comt_inhibitor + carbidopa_levodopa + anticholinergic" \
    --p-n-jobs 2 \
    --o-visualization complete_study/diversity/permanova_with_confounders.qzv

Anteriormente, no observábamos un valor p significativo para el efecto de la enfermedad de Parkinson en la diversidad beta. Sin embargo, al añadir ciertas covariables, podríamos descubrir que confundían una relación.

Sin embargo, la mayor parte de nuestra varianza permanece sin explicar. La composición del microbioma se ve afectada por muchos factores, y este es un estudio de cohorte no controlado, por lo que no esperaríamos que una sola variable explicara la mayor parte de la varianza.

#### Reducción de dimensionalidad

Si queremos mostrar visualmente esta separación entre muestras, no podemos simplemente graficar la matriz de distancias UniFrac completa, ya que tiene más de 100 dimensiones. En su lugar, podemos usar la **reducción de dimensionalidad** para comprimir nuestros datos en unas pocas dimensiones que expliquen la mayor parte de la varianza. Existen varios tipos de reducción de dimensionalidad (como UMAP y tSNE), pero el método preferido para las comunidades de microbiomas es el Análisis de Coordenadas Principales (PCoA). Esto se debe a que el PCoA es lineal y, por lo tanto, preserva la estructura global de los datos y es reproducible.

Ya ejecutamos un PCoAs para cada métrica de distancia y podemos verlos si los descargamos. `weighted_unifrac_emperor.qzv`, `unweighted_unifrac_emperor.qzv`, `bray_curtis_emperor.qzv`, or `jaccard_emperor.qzv`

---
# Clasificación taxonómica

### De ASV a tabla de recuento taxonómico

Podemos aprender mucho de las métricas de diversidad, alfa y beta. Pero para analizar a fondo los datos, necesitamos saber qué microbios hay en cada muestra 🦠. Para ello, clasificaremos las lecturas en QIIME2 utilizando un clasificador bayesiano preentrenado. Varios clasificadores de este tipo están disponibles en https://docs.qiime2.org/2024.5/data-resources/

Desde el plugin `feature-classifier` de QIIME2, usaremos la acción `classifiy-sklearn`, que toma como entrada los ASV y un clasificador scikit-learn preentrenado. Utilizamos un clasificador bayesiano ingenuo específico para la región 16S V4 (515f-806r) porque es la región que secuenciamos.

In [ ]:
# run a classifier on our 10-sample mini dataset
!qiime feature-classifier classify-sklearn \
    --i-reads dada/representative_sequences.qza \
    --i-classifier ncbi-refseq-16S-v4-classifier.qza \
    --p-n-jobs 2 \
    --o-classification taxa.qza

Ahora que hemos clasificado las lecturas, podemos visualizar el desglose taxonómico de nuestras muestras.

In [ ]:
!qiime taxa barplot \
    --i-table dada/table.qza \
    --i-taxonomy taxa.qza \
    --m-metadata-file data/metadata.tsv \
    --o-visualization taxa_barplot.qzv

Ahora, podemos usar ```table.qza```, que contiene nuestras lecturas, y ```taxa.qza```, que contiene clasificaciones taxonómicas para las lecturas, y colapsar los datos al nivel de género.

In [ ]:
!qiime taxa collapse \
    --i-table dada/table.qza \
    --i-taxonomy taxa.qza \
    --p-level 6 \
    --o-collapsed-table genus.qza

### Exportando su tabla de taxones

Las tablas de características taxonómicas en QIIME2 se pueden exportar como archivos .biom, que luego se pueden convertir a archivos .tsv si se desea. Esto es importante, ya que algunos métodos de análisis no disponibles en QIIME2 podrían requerir los formatos .biom o .tsv.

In [ ]:
#instalar biom-format
!pip install biom-format pandas

In [ ]:
!qiime tools export \
    --input-path genus.qza \
    --output-path exported

!biom convert -i exported/feature-table.biom -o genus.tsv --to-tsv

Echemos un vistazo a los resultados. 🔭

In [ ]:
abundances = pd.read_table("genus.tsv", skiprows=1, index_col=0)
abundances

Esto es más fácil de interpretar visualizando los resultados. Podemos usar el archivo que acabamos de exportar de QIIME2 para crear una visualización con cualquier herramienta que prefiramos, como seaborn o plotnine.

A continuación, se muestra un ejemplo de creación de una visualización (un mapa de calor) en seaborn:

In [ ]:
import numpy as np
import seaborn as sns

In [ ]:
abund_to_plot = abundances.copy()
abund_to_plot.index = abund_to_plot.index.str.split(";").str[5]          # Utilice únicamente el nombre del género
abund_to_plot = abund_to_plot[~abund_to_plot.index.isin(["g__", "__"])]  # eliminar géneros no clasificados
abund_to_plot = abund_to_plot.sample(50, axis=0)                         # utilizar 50 géneros aleatorios (filas)

# Hagamos una transformación logarítmica centrada: log x_i - log mean(x)
transformed = abund_to_plot.apply(
    lambda xs: np.log(xs + 0.5) - np.log(xs.mean() + 0.5),
    axis=0)

sns.clustermap(transformed.T, cmap="magma", xticklabels=True, figsize=(16, 6))

Ahora, nuestros datos empiezan a ser interpretables. Cada fila representa una muestra y cada columna un género bacteriano. Los valores de la tabla son "conteos", o el número de veces que se detectó un género en una muestra determinada. Podemos usar datos de abundancia relativa para comprobar hipótesis, pero requiere métodos estadísticos especiales porque es ___composicional___.

Analicemos qué podemos hacer al respecto en la **presentación**

---
# Análisis de abundancia diferencial

Veamos las abundancias en el conjunto de datos _completo_, para que pueda ver las distribuciones de abundancia.

Vamos a cambiar a Python para mostrarte estas transformaciones, por lo que necesitaremos algunos paquetes más. 📦

In [ ]:
import scipy.stats as stats
import matplotlib.pyplot as plt

### Conteos de abundancia relativa:

Antes de realizar cualquier prueba, debemos dividir el número de géneros entre el número total de muestras. Esto nos da la _abundancia relativa_.

In [ ]:
complete_abundances = pd.read_table('complete_study/genus.tsv', skiprows=1, index_col=0)
complete_abundances

In [ ]:
rel_abund = complete_abundances.divide(complete_abundances.sum(axis=0), axis=1)     # divide all values in a row by the sum of the row
rel_abund

### Preprocesamiento

¿Existen filas que tengan todo cero?

In [ ]:
rows_with_zeros = rel_abund.eq(0).all(axis=1)
count_row_zeros = rows_with_zeros.sum()
count_row_zeros

¿Cuántas filas (géneros) no están presentes en la mayoría de las muestras (cero en >80% de las muestras)?

In [ ]:
# identificar filas dispersas
num_zeros_row = rel_abund.eq(0).sum(axis=1)
num_cols = rel_abund.shape[1]
sparse_rows = num_zeros_row > (0.8 * num_cols)
print(sparse_rows.sum())

# eliminar filas dispersas
abund_filt = rel_abund[~sparse_rows]

# transponer para hacer filas de muestras en lugar de géneros (que coincida con el formato de nuestros metadatos)
abund_filt = abund_filt.T
abund_filt

Guarde los nombres de las columnas de géneros en una lista y luego fusione las tablas de abundancia y metadatos.

In [ ]:
genus_columns = abund_filt.columns                                                            # obtener una lista de nombres de columnas de abundancia
complete_metadata.set_index('id', inplace=True)                                               # cambiar el índice (nombres de filas) de los metadatos a la columna 'id'
merged_rel_abund = complete_metadata.merge(abund_filt, left_index=True, right_index=True)     # fusionar los dos marcos de datos por su índice (nombres de fila)

In [ ]:
# Hacer un histograma de Faecalibacterium dividido por estado de PD

sns.displot(data=merged_rel_abund,
            x='k__Bacteria;p__Firmicutes;c__Clostridia;o__Eubacteriales;f__Oscillospiraceae;g__Faecalibacterium',
            hue='parkinson_disease')

plt.xlabel('Faecalibacterium relative abundance')
plt.ylabel('Number of samples')
plt.show()

### 1. Pruebas no paramétricas

Si quisiéramos comprobar si _Faecalibacterium_ específicamente era diferencialmente abundante en PD, podríamos hacer esto:

In [ ]:
# Dividir la variable de interés en 2 listas por estado de PD
faecali_pd = merged_rel_abund[merged_rel_abund['parkinson_disease'] == 'Yes']['k__Bacteria;p__Firmicutes;c__Clostridia;o__Eubacteriales;f__Oscillospiraceae;g__Faecalibacterium']
faecali_hc = merged_rel_abund[merged_rel_abund['parkinson_disease'] == 'No']['k__Bacteria;p__Firmicutes;c__Clostridia;o__Eubacteriales;f__Oscillospiraceae;g__Faecalibacterium']

u_stat, p_value = stats.mannwhitneyu(faecali_pd, faecali_hc, alternative='two-sided')

print(f"U-statistic: {u_stat}")
print(f"p-value: {p_value}")

Veríamos que sí, si solo realizamos una prueba, parece que la abundancia relativa media de _Faecalibacterium_ es significativamente diferente entre PD y controles sanos (con p < 0.05)

### 2. Normalización + pruebas paramétricas:

Antes de que podamos realizar una transformación CLR, tenemos que agregar un pseudocount, porque log(0) no existe.

In [ ]:
# realizar un seguimiento de qué valores son distintos de cero
abund_nonzero = abund_filt > 0

# añadir un pseudoconteo
abund_filt_pseudo = abund_filt + 0.0000001
abund_filt_pseudo

Ahora que tenemos pseudoconteos, podemos realizar la transformación CLR. Aquí, usamos la función gmean de scipy.stats.mstats y calculamos el CLR manualmente.

La transformación de razón logarítmica centrada (clr) divide cada parte de la composición entre la media geométrica de todas las partes.

In [ ]:
# crear una nueva columna para la media geometrica
abund_clr = abund_filt_pseudo.copy()
abund_clr['gmean'] = abund_clr.apply(stats.mstats.gmean, axis=1, nan_policy='omit')

# actual CLR
abund_clr = np.log(abund_clr.divide(abund_clr['gmean'], axis=0))

# Verifique gmean para asegurarse de que CLR se realizó correctamente
# gmean debe ser todo ceros (because log(x)/log(x) = log(x/x) = log(1) = 0)
abund_clr

In [ ]:
# drop gmean
abund_clr.drop(columns='gmean', inplace=True)

Ahora, veamos cómo afectó la transformación a nuestros datos. Podemos crear el mismo gráfico que antes.

In [ ]:
sns.displot(data=abund_clr,
            x='k__Bacteria;p__Firmicutes;c__Clostridia;o__Eubacteriales;f__Oscillospiraceae;g__Faecalibacterium')
plt.xlabel('Faecalibacterium CLR')
plt.ylabel('Number of samples')
plt.show()

Ahora, tenemos una distribución bimodal debido a la presencia de ceros. En este punto, la opción más conservadora es eliminar estos valores, ya que no podemos estar seguros de si son ceros verdaderos.

In [ ]:
# descartar las muestras que fueron cero (hacer NA para que no se grafiquen)
abund_clr_nonzero = abund_clr[abund_nonzero]

In [ ]:
# Ahora intenta trazar de nuevo
sns.displot(data=abund_clr_nonzero,
            x='k__Bacteria;p__Firmicutes;c__Clostridia;o__Eubacteriales;f__Oscillospiraceae;g__Faecalibacterium')
plt.xlabel('Faecalibacterium CLR')
plt.ylabel('Number of samples')
plt.show()

Ahora nuestros datos son _bastante normales_ (aunque aún un poco distorsionados), pero probablemente seguros para usar con pruebas paramétricas. Esto significa que estamos listos para fusionarlos de nuevo con los metadatos.

In [ ]:
merged_clr = complete_metadata.merge(abund_clr_nonzero, left_index=True, right_index=True)     # fusionar los dos marcos de datos por su índice (nombres de fila)
merged_clr

El código para realizar una prueba t es muy similar al de una prueba U de Mann-Whitney, pero también necesitamos eliminar los NA.

Podemos lograr esto añadiendo `dropna()` al final de nuestra asignación de grupo.

In [ ]:
# Cómo haríamos una prueba t
faecali_clr_pd = merged_clr[merged_clr['parkinson_disease'] == 'Yes']['k__Bacteria;p__Firmicutes;c__Clostridia;o__Eubacteriales;f__Oscillospiraceae;g__Faecalibacterium'].dropna()
faecali_clr_hc = merged_clr[merged_clr['parkinson_disease'] == 'No']['k__Bacteria;p__Firmicutes;c__Clostridia;o__Eubacteriales;f__Oscillospiraceae;g__Faecalibacterium'].dropna()

t_stat, p_val = stats.ttest_ind(faecali_clr_pd, faecali_clr_hc, equal_var=False)

print(f"T-stat: {t_stat}")
print(f"p-value: {p_val}")

En personas con cantidades _medibles_ de Faecalibacterium, las personas con EP tienen niveles significativamente menores de Faecalibacterium que las personas sin EP.

Observe cómo, al recortar nuestros datos, ahora estamos evaluando una pregunta diferente.

### Pruebas múltiples y corrección FDR:

Con tantos datos interesantes, sería bastante tedioso probar las variables una por una. Por eso creamos anteriormente la lista `genus_column`.

In [ ]:
# Crea una lista vacía para almacenar los resultados de las pruebas
t_test_results = []

# Recorra cada columna de género y realice la prueba t
for genus in genus_columns:
    # Separar los datos de los grupos “Sí” y “No” de la enfermedad de Parkinson
    pd_group = merged_clr[merged_clr['parkinson_disease'] == 'Yes'][genus].dropna()
    hc_group = merged_clr[merged_clr['parkinson_disease'] == 'No'][genus].dropna()

    # Realizar la prueba t
    if len(pd_group) > 0 and len(hc_group) > 0:  # Realice la prueba solo si hay valores en ambos grupos
        t_stat, p_val = stats.ttest_ind(pd_group, hc_group, equal_var=False)

        # Almacenar los resultados
        t_test_results.append({
            'genus': genus,
            't_stat': t_stat,
            'p_val': p_val,
            'n_pd': len(pd_group),
            'n_hc': len(hc_group)
        })

# Convertir los resultados en un DataFrame
ttest_results_df = pd.DataFrame(t_test_results)

In [ ]:
# Eche un vistazo a nuestro marco de datos de resultados
ttest_results_df

¿Cómo se ve nuestra distribución de valor p?

In [ ]:
sns.histplot(data=ttest_results_df, x='p_val', bins=13)
plt.xlabel('p-value')
plt.ylabel('number of t-tests')
plt.show()

¿Se ve bien? Esperamos una desviación hacia la izquierda, dados tanto los verdaderos como los falsos positivos.

Cuando disponemos de una lista de valores p, podemos aplicar fácilmente la corrección FDR. Utilizaremos la corrección de Benjamini-Hochberg (BH), que es menos conservadora que la de Bonferroni. El valor p corregido por BH, q, se calcula en función de la lista completa de valores q.

In [ ]:
from statsmodels.stats.multitest import multipletests

In [ ]:
# realizar la corrección FDR (esto devuelve los valores q en el orden en que fueron proporcionados)
qvals = multipletests(ttest_results_df['p_val'], method='fdr_bh')[1]

# añadir valores q al dataframe
ttest_results_df['q_val'] = qvals

# mostrar el nuevo dataframe, pero ordenar por valor q
ttest_results_df.sort_values(by='q_val')

¡Genial! Separemos solo los resultados significativos y creamos una columna con nombres taxonómicos más fáciles de representar.

In [ ]:
ttest_signif = ttest_results_df[ttest_results_df['q_val'] < 0.1]
ttest_signif.loc[:, 'genus_plot'] = ttest_signif['genus'].str.split(';').apply(lambda x: ';'.join(x[-2:])) # tomar sólo los 2 últimos niveles taxonómicos
ttest_signif

¿Cómo podemos visualizar esta información? Aquí, he trazado el estadístico T para cada impacto significativo, coloreado por el valor q.

In [ ]:
# Ordenar por estadística t absoluta para una mejor visualización
ttest_signif = ttest_signif.sort_values(by='t_stat', key=abs, ascending=False)

# Crea una paleta de colores personalizada que vaya de claro a oscuro según los valores q.
norm = plt.Normalize(0.0, 0.1)
cmap = plt.cm.Blues_r  # color map
bar_colors = cmap(norm(ttest_signif['q_val'])).tolist()

# Plot
fig, ax = plt.subplots(figsize=(10, 6))
sns.barplot(data=ttest_signif, x='t_stat', y='genus_plot', palette=bar_colors, hue='genus_plot', ax=ax, legend=False)

# Hacer etiquetas
ax.set_title('Differentially Abundant Genera in PD patients')
ax.set_xlabel('T-Statistic')
ax.set_ylabel('Genus')

# crear una clave para los valores q
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)  # crear objeto ScalarMappable
sm.set_array([])  # Matriz vacía para ScalarMappable
cbar = plt.colorbar(sm, ax=ax)
cbar.set_label('q-value')

# Mostrar la figura
plt.tight_layout()
plt.show()

---
# Ejercicio: Plantar un árbol

Una visualización a la que no dedicamos mucho tiempo fue el árbol filogenético de nuestros ASV. ¡Vamos a cambiarlo! En el paso anterior, vimos que hay géneros que aparecen en múltiples poblaciones. Pero, ¿son los organismos de ese género realmente los mismos?

Anotemos el árbol con nuestras clasificaciones taxonómicas y abundancias. Usaremos de nuevo el complemento empress, pero esta vez con la opción `community-plot`. Les preparé una plantilla del comando. ¿Pueden indicar qué debe ir en los espacios vacíos?

**PREGUNTAS:**

1) ¿Son algunas ramas del árbol más largas de lo esperado? ¿Observas algo interesante o sospechoso en la identidad taxonómica de estas ramas?

2) ¿Puedes encontrar ejemplos de filos polifiléticos (es decir, grupos de ASV del mismo filo que se encuentran en diferentes lugares del árbol, mostrando diferentes ancestros comunes)? ¿Qué ocurre con los taxones polifiléticos en niveles taxonómicos inferiores, como a nivel de familia o género? ¿Por qué crees que existen estos patrones?

Nota: Es necesario entregar las respuestas de las preguntas, y el nuevo arbol anotado para poder acreditar el curso. Todo deberá estar en un archivo pdf y se deberá enviar por correo a microbiomas.salud2025@gmail.com **Asunto Taller 16S**

In [ ]:
# Esto no se ejecutará hasta que completes los espacios [VACÍOS] con los archivos correctos ;)

!qiime empress community-plot \
    --i-tree [EMPTY] \
    --i-feature-table dada/table.qza \
    --m-sample-metadata-file [EMPTY] \
    --m-feature-metadata-file taxa.qza \
    --o-visualization community-tree-viz.qzv